1. Импортируем библиотеки

In [61]:
import pandas as pd
import scipy
import scipy.stats

2. Загружаем файл csv и смотрим изначально загруженный вариант

In [62]:
data_raw=pd.read_csv(r'M-TEX DATA/М.Тех_Данные_к_ТЗ_DS.csv',sep=',', encoding='cp1251', quoting=3 )
display(data_raw.head())

,"""Количество больничных дней","""""Возраст""""","""""Пол"""""""
0,"""5",39,"""""Ж"""""""
1,"""4",54,"""""М"""""""
2,"""4",26,"""""М"""""""
3,"""5",42,"""""М"""""""
4,"""4",29,"""""М"""""""


3. Редактируем данный так, чтобы их можно было дальше обрабатывать

In [63]:
new_columns=[]
for i in range(len(data_raw.columns)):
    #print(data_raw.columns[i])
    x=data_raw.columns[i].replace('"', "")
    #print(x)
    new_columns.append(x)
print(new_columns)

data_raw.columns=new_columns

new_ill_days=[]
for i in range(len(data_raw['Количество больничных дней'])):
    #print(data_raw['Количество больничных дней'][i])
    x=data_raw['Количество больничных дней'][i].replace('"', "")
    #print(x)
    new_ill_days.append(x)
#print(new_ill_days)
data_raw['Количество больничных дней']=new_ill_days

new_sexes=[]
for i in range(len(data_raw['Пол'])):
    #print(data_raw['Пол'][i])
    x=data_raw['Пол'][i].replace('"', "")
    #print(x)
    new_sexes.append(x)
#print(new_sexes)
data_raw['Пол']=new_sexes

display(data_raw.head())

['Количество больничных дней', 'Возраст', 'Пол']


,Количество больничных дней,Возраст,Пол
0,5,39,Ж
1,4,54,М
2,4,26,М
3,5,42,М
4,4,29,М


4. Проверим данные на Nan и проверим однородность форматов по столбцам

In [64]:
print('-------------------------------------------------------------------------------')
print(pd.DataFrame(data_raw['Количество больничных дней']).applymap(type).eq(str).all())
print(pd.DataFrame(data_raw['Возраст']).applymap(type).eq(int).all())
print(pd.DataFrame(data_raw['Пол']).applymap(type).eq(str).all())
print('-------------------------------------------------------------------------------')
print(pd.DataFrame(data_raw).isnull().any())
print('-------------------------------------------------------------------------------')
print(pd.DataFrame(data_raw).isnull().sum())
print('-------------------------------------------------------------------------------')

-------------------------------------------------------------------------------
Количество больничных дней    True
dtype: bool
Возраст    True
dtype: bool
Пол    True
dtype: bool
-------------------------------------------------------------------------------
Количество больничных дней    False
Возраст                       False
Пол                           False
dtype: bool
-------------------------------------------------------------------------------
Количество больничных дней    0
Возраст                       0
Пол                           0
dtype: int64
-------------------------------------------------------------------------------


In [65]:
df_raw=pd.DataFrame(data_raw)

I. Задача о зависимости числа больничных дней от пола работника

1.Введём dummies для пола

In [66]:
df_sexes=pd.get_dummies(df_raw, columns=['Пол'])
df_sexes['Количество больничных дней']=df_sexes['Количество больничных дней'].astype(float)
display(df_sexes.head())

,Количество больничных дней,Возраст,Пол_Ж,Пол_М
0,5.0,39,1,0
1,4.0,54,0,1
2,4.0,26,0,1
3,5.0,42,0,1
4,4.0,29,0,1


2. Выделим случаи, где более 2-х пропущенных дней

In [70]:
df_sexes=df_sexes[df_sexes['Количество больничных дней']>2]
print('Всего случаев пропусков более 2-х дней:',len(df_sexes))

Всего случаев пропусков более 2-х дней: 151


3. Создадим данные о работниках, болевших более 2-х дней, male и female работниках двумя отдельными dataset-ами

In [71]:
df_male=df_sexes[df_sexes['Пол_М']==1]
print('Число male:', len(df_male))
df_female=df_sexes[df_sexes['Пол_Ж']==1]
print('Число female',len(df_female))
print("Всего людей:",len(df_male)+len(df_female))

Число male: 84
Число female 67
Всего людей: 151


4. Подсчитаем для male и female их число и mean, var и std

In [72]:
sick_leaves_male_mean=df_male['Количество больничных дней'].mean()
sick_leaves_male_var=df_male['Количество больничных дней'].var()
sick_leaves_male_std=df_male['Количество больничных дней'].std()
n_males=len(df_male['Количество больничных дней'])
print(f'MALES: n_males={n_males},sick_leaves_male_mean={sick_leaves_male_mean:.3},sick_leaves_male_var={sick_leaves_male_var:.3},sick_leaves_male_std={sick_leaves_male_std:.3}')
sick_leaves_female_mean=df_female['Количество больничных дней'].mean()
sick_leaves_female_var=df_female['Количество больничных дней'].var()
sick_leaves_female_std=df_female['Количество больничных дней'].std()
n_females=len(df_female['Количество больничных дней'])
print(f'FEMALES: n_females={n_females},sick_leaves_female_mean={sick_leaves_female_mean:.3},sick_leaves_female_var={sick_leaves_female_var:.3},sick_leaves_female_std={sick_leaves_female_std:.3}')

MALES: n_males=84,sick_leaves_male_mean=4.33,sick_leaves_male_var=2.18,sick_leaves_male_std=1.48
FEMALES: n_females=67,sick_leaves_female_mean=4.24,sick_leaves_female_var=1.61,sick_leaves_female_std=1.27


5. Выдвинем нулевую гипотезу - количество больничных дней больше 2-х не зависит от пола работника. Применим дисперсионный анализ: обе выборки (males и females) принадлежат одной генеральной совокупности. Применим двусторонний критерий Фишера.
Обозначения:
p - уровень значимости
F_sexes - значение полученной F  статистики (делим var males на var females)
F_crit_right - критическое значение справа
F_crit_left  - критическое значение слева
Если F_sexes попадает между F_crit_left и F_crit_right, то принимаем нулевую гипотезу при данном уровне значимости

In [73]:
p=0.2
F_sexes=sick_leaves_male_var/sick_leaves_female_var
F_crit_right=scipy.stats.f.ppf(1-p/2, n_males-1, n_females-1)
F_crit_left=1/(scipy.stats.f.ppf(1-p/2, n_females-1, n_males-1))
print(f'p = {p}, F_sexes={F_sexes:.3}, F_crit_left={F_crit_left:.3}, F_crit_right={F_crit_right:.3}')

p = 0.2, F_sexes=1.35, F_crit_left=0.743, F_crit_right=1.36


In [74]:
print('----------------------------------------------------------------------')
print('ЗАДАЧА 1. РЕШЕНИЕ:')
print('----------------------------------------------------------------------')
print('НУЛЕВАЯ ГИПОТЕЗА: ЧИСЛО БОЛЬНИЧНЫХ ДНЕЙ БОЛЬШЕ 2-Х НЕ ЗАВИСИТ ОТ ПОЛА РАБОТНИКА ')
print('----------------------------------------------------------------------')
print('ОТВЕТ:')
if F_sexes<=F_crit_right and F_sexes>=F_crit_left:
    print(f'НУЛЕВАЯ ГИПОТЕЗА ПРИНИМАЕТСЯ при уровне значимости p={p}, ЧИСЛО БОЛЬНИЧНЫХ ДНЕЙ БОЛЬШЕ 2-Х  НЕ ЗАВИСИТ ОТ ПОЛА РАБОТНИКА')
else:
    print('НУЛЕВАЯ ГИПОТЕЗА ОТВЕРГАЕТСЯ при уровне значимости p={p}')
print('----------------------------------------------------------------------')

----------------------------------------------------------------------
ЗАДАЧА 1. РЕШЕНИЕ:
----------------------------------------------------------------------
НУЛЕВАЯ ГИПОТЕЗА: ЧИСЛО БОЛЬНИЧНЫХ ДНЕЙ БОЛЬШЕ 2-Х НЕ ЗАВИСИТ ОТ ПОЛА РАБОТНИКА 
----------------------------------------------------------------------
ОТВЕТ:
НУЛЕВАЯ ГИПОТЕЗА ПРИНИМАЕТСЯ при уровне значимости p=0.2, ЧИСЛО БОЛЬНИЧНЫХ ДНЕЙ БОЛЬШЕ 2-Х  НЕ ЗАВИСИТ ОТ ПОЛА РАБОТНИКА
----------------------------------------------------------------------


II. Задача о зависимости числа больничных дней от возраста работника

1. Создадим данные о работниках, болевших более 2-х дней, старше 35-ти лет и младшэ 35-ти лет, двумя отдельными dataset-ами

In [75]:
df_ages=df_sexes
df_young=df_ages[df_ages['Возраст']<=35]
print("Число людей, младше 35-ти включительно:",len(df_young))
df_old=df_ages[df_ages['Возраст']>35]
print("Число людей, старше 35-ти:",len(df_old))
print("Всего людей:",len(df_young)+len(df_old))

Число людей, младше 35-ти включительно: 36
Число людей, старше 35-ти: 115
Всего людей: 151


In [ ]:
2. Подсчитаем для "старше 35-ти" и "до 35-ти включительно" их число и mean, var и std

In [76]:
sick_leaves_young_mean=df_young['Количество больничных дней'].mean()
sick_leaves_young_var=df_young['Количество больничных дней'].var()
sick_leaves_young_std=df_young['Количество больничных дней'].std()
n_young=len(df_young['Количество больничных дней'])
print(f'До 35-ти включительно: n_young={n_young},sick_leaves_young_mean={sick_leaves_young_mean:.3},sick_leaves_young_var={sick_leaves_young_var:.3},sick_leaves_young_std={sick_leaves_young_std:.3}')

sick_leaves_old_mean=df_old['Количество больничных дней'].mean()
sick_leaves_old_var=df_old['Количество больничных дней'].var()
sick_leaves_old_std=df_old['Количество больничных дней'].std()
n_old=len(df_old['Количество больничных дней'])
print(f'Старше 35-ти: n_old={n_old},sick_leaves_old_mean={sick_leaves_old_mean:.3},sick_leaves_old_var={sick_leaves_old_var:.3},sick_leaves_old_std={sick_leaves_old_std:.3}')


До 35-ти включительно: n_young=36,sick_leaves_young_mean=4.44,sick_leaves_young_var=2.14,sick_leaves_young_std=1.46
Старше 35-ти: n_old=115,sick_leaves_old_mean=4.24,sick_leaves_old_var=1.85,sick_leaves_old_std=1.36


3. Выдвинем нулевую гипотезу - количество больничных дней больше 2-ч не зависит от возраста работника. Применим дисперсионный анализ: обе выборки (males и females) принадлежат одной генеральной совокупности. Применим двусторонний критерий Фишера.
Обозначения:
p - уровень значимости
F_ages - значение полученной F  статистики (делим var young на var old)
F_crit_right - критическое значение справа
F_crit_left  - критическое значение слева
Если F_ages попадает между F_crit_left и F_crit_right, то принимаем нулевую гипотезу при данном уровне значимости

In [77]:
p=0.2
F_ages=sick_leaves_young_var/sick_leaves_old_var
F_crit_right=scipy.stats.f.ppf(1-p/2, n_young-1, n_old-1)
F_crit_left=1/(scipy.stats.f.ppf(1-p/2, n_old-1, n_young-1))
print(f'p = {p}, F_ages={F_ages:.3}, F_crit_left={F_crit_left:.3}, F_crit_right={F_crit_right:.3}')

p = 0.2, F_ages=1.16, F_crit_left=0.685, F_crit_right=1.39


In [78]:
print('----------------------------------------------------------------------')
print('ЗАДАЧА 2. РЕШЕНИЕ:')
print('----------------------------------------------------------------------')
print('НУЛЕВАЯ ГИПОТЕЗА: ЧИСЛО БОЛЬНИЧНЫХ ДНЕЙ БОЛЬШЕ 2-Х НЕ ЗАВИСИТ ОТ ВОЗРАСТА РАБОТНИКА ')
print('----------------------------------------------------------------------')
print('ОТВЕТ:')
if F_ages<=F_crit_right and F_ages>=F_crit_left:
    print(f'НУЛЕВАЯ ГИПОТЕЗА ПРИНИМАЕТСЯ при уровне значимости p={p}, ЧИСЛО БОЛЬНИЧНЫХ ДНЕЙ БОЛЬШЕ 2-Х НЕ ЗАВИСИТ ОТ ВОЗРАСТА РАБОТНИКА')
else:
    print('НУЛЕВАЯ ГИПОТЕЗА ОТВЕРГАЕТСЯ при уровне значимости p={p}')
print('----------------------------------------------------------------------')

----------------------------------------------------------------------
ЗАДАЧА 2. РЕШЕНИЕ:
----------------------------------------------------------------------
НУЛЕВАЯ ГИПОТЕЗА: ЧИСЛО БОЛЬНИЧНЫХ ДНЕЙ БОЛЬШЕ 2-Х НЕ ЗАВИСИТ ОТ ВОЗРАСТА РАБОТНИКА 
----------------------------------------------------------------------
ОТВЕТ:
НУЛЕВАЯ ГИПОТЕЗА ПРИНИМАЕТСЯ при уровне значимости p=0.2, ЧИСЛО БОЛЬНИЧНЫХ ДНЕЙ БОЛЬШЕ 2-Х НЕ ЗАВИСИТ ОТ ВОЗРАСТА РАБОТНИКА
----------------------------------------------------------------------
